## Predicting Log P value given SMILES using RDKit at Zinc Dataset 
Given the Zinc Dataset, we'll take the SMILES strings and predict the Log P values using RDKit and see the performance 

In [1]:
from rdkit import Chem 
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole 
from rdkit.Chem import Descriptors 
from rdkit.Chem import AllChem 
from rdkit import DataStructs
import numpy as np 
from rdkit.Chem import MolFromInchi
from rdkit.Chem import rdMolDescriptors
import pandas as pd 


### Experiment 1: 
- Take the first 100 original SMILES and predict their log P value 

In [ ]:
data= pd.read_csv("enhanced_molecules_top1000 (2).csv", usecols=[0,1],nrows=100) # take in the first col and first 100 smiles 
data.head() 

,smiles,logP
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956


In [25]:
smiles= np.array(data["smiles"])    # (100,)
logP_gt= np.array(data["logP"])     # (100,)
print(smiles[0])
print(logP_gt[0])

CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1

5.0506


In [ ]:
# iterate through the entire smiles array to feed into RDKit 
# combine RDKit results into one array and calculate the RMSE with ground_truth
pred_logP= [] # (100,)
for i in range(0,len(smiles)):
    mol =  Chem.MolFromSmiles(smiles[i])    # generate molecule in RDKit 
    value= Descriptors.MolLogP(mol)     # predict log P value 
    pred_logP.append(value)     # append to the list 

pred_logP= np.array(pred_logP)  # convert to np array 
print(pred_logP[:5])    # print out predictions 

[5.0506  3.1137  4.96778 4.00022 3.60956]


In [34]:
# calculate error metrics 
mse= np.mean( (pred_logP-logP_gt)**2 ,axis=0)
print("Mean Squared Error:",mse)

rmse= np.sqrt (mse)
print("Root Mean Squared Error:",rmse)

Mean Squared Error: 4.782392200704608e-30
Root Mean Squared Error: 2.1868681260434085e-15


#### Experiment 2: 
Same set up as experiment 1, given original canonical and isomeric SMILES, predict log P values, but we use the entire data set 

In [36]:
data= pd.read_csv("enhanced_molecules_top1000 (2).csv", usecols=[0,1]) # take in the first col and first 100 smiles 
logP_gt= np.array(data["logP"])     
smiles= np.array(data["smiles"]) 
pred_logP= [] 
for i in range(0,len(smiles)):
    mol =  Chem.MolFromSmiles(smiles[i])   
    value= Descriptors.MolLogP(mol)     
    pred_logP.append(value)     

pred_logP= np.array(pred_logP)   # (1000, )
print(pred_logP[:5])

[5.0506  3.1137  4.96778 4.00022 3.60956]


In [37]:
mse= np.mean( (pred_logP-logP_gt)**2 ,axis=0)
print("1000 Smiles Mean Squared Error:",mse)

rmse= np.sqrt (mse)
print("1000 Smiles Root Mean Squared Error:",rmse)

1000 Smiles Mean Squared Error: 3.44390984429787e-30
1000 Smiles Root Mean Squared Error: 1.8557774231566325e-15


In [38]:
# convert into one data file 
data = pd.DataFrame(
    {
        "smiles":smiles, 
        "logP": logP_gt,
        "RDKit logP": pred_logP 
    }
)
data.to_csv("logP_RDKit_predicted.csv", index=False)


## Experiment Helper Methods 

In [2]:
def pred_logP (smiles_lst): 
    '''  
    Arguments smiles_lst: input list of SMILES strings to read. 
    Return: list of predicted log P value corresponding to each SMILES 
    '''
    pred = [] 
    for i in range(0, len(smiles_lst)): 
        mol= Chem.MolFromSmiles(smiles_lst[i])
        value = Descriptors.MolLogP(mol) 
        pred.append(value)
    return pred

def pred_QED (smiles_lst): 
    '''  
    Arguments smiles_lst: input list of SMILES strings to read. 
    Return: list of predicted QED value corresponding to each SMILES 
    '''
    pred = [] 
    for i in range(0, len(smiles_lst)): 
        mol= Chem.MolFromSmiles(smiles_lst[i])
        value = Descriptors.qed(mol) 
        pred.append(value)
    return pred

def pred_SAS (smiles_lst): 
    '''  
    Arguments smiles_lst: input list of SMILES strings to read. 
    Return: list of predicted SAS value corresponding to each SMILES 
    '''
    from rdkit.Chem import RDConfig
    import os
    import sys
    sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
    import sascorer
    
    pred = [] 
    for i in range(0, len(smiles_lst)): 
        mol= Chem.MolFromSmiles(smiles_lst[i])
        value = sascorer.calculateScore(mol) 
        pred.append(value)
    return pred


In [3]:
# error calculation metrics 

# root mean squared error 
def RMSE(gt,pred): 
    ''' 
    Arguments: 
        gt: ground truth  (n_row, )
        pred: predicted array   (n_row, )
    Return: total RMSE error 
    ''' 
    gt= np.array(gt)
    pred = np.array(pred)
    mse = np.mean( (gt-pred)**2, axis=0)
    return np.sqrt(mse)
    
# mean absolute error 
def MAE(gt, pred): 
    ''' 
    Arguments: 
        gt: ground truth  (n_row, )
        pred: predicted array   (n_row, )
    Return: total RMSE error 
    ''' 
    gt= np.array(gt)
    pred = np.array(pred)
    return np.mean(np.abs(gt-pred), axis=0) 



#### Experiment 3: 
Log P predictions with random(non-canonical), non-isomeric, kekule SMILES  
*Note: ZINC dataset uses canonical and isomeric SMILES* 

In [ ]:
data= pd.read_csv("enhanced_molecules_top1000 (2).csv", usecols=[1,5,6,7,8])


In [29]:
logP_gt= data["logP"]
# all testing SMILES 
smiles_dic= {
    "ran_1": data["Random_SMILES_1"], 
    "ran_2" : data["Random_SMILES_2"], 
    "non_iso": data["NonIsomeric_SMILES"], 
    "kekule" : data["Kekule_SMILES"]
}

# running experiment to get prediction output and error 
all_output = []
for smiles in smiles_dic: 
    out= pred_logP(smiles_dic[smiles])
    print(f"{smiles} RMSE Error: {RMSE(logP_gt, out)}")
    print (f"{smiles} MAE Error: {MAE(logP_gt, out)}")
    all_output.append(out)

# append the existing csv file 
new_data= pd.DataFrame({
    "Log P" : logP_gt
})
key_index = list(smiles_dic.keys())   # convert keys to index for faster append 
for i in range(0, len(key_index)):
    new_data[key_index[i]] = all_output[i]
    

ran_1 RMSE Error: 1.8665324996562116e-15
ran_1 MAE Error: 1.4125697139766303e-15
ran_2 RMSE Error: 1.9039164274386363e-15
ran_2 MAE Error: 1.455361872682026e-15
non_iso RMSE Error: 1.8557508552294186e-15
non_iso MAE Error: 1.4117301078142575e-15
kekule RMSE Error: 1.8578410870503903e-15
kekule MAE Error: 1.4157824218541393e-15


In [ ]:
# append to new data 
new_data.to_csv("non_canonical_RDKit_logP.csv", index=False)

#### Experiment 4: Predict QED for Canonincal SMILES 
- QED Score predictions are not as good as log P, but still very good compared with GPT outputs 

In [6]:
data= pd.read_csv("enhanced_molecules_top1000 (2).csv", usecols=[0,2])
smiles= data["smiles"]
qed_gt= data["qed"]
pred = pred_QED(smiles)
print("QED Score RMSE:", RMSE(qed_gt,pred))
print("QED Score MAE:", MAE(qed_gt,pred))

QED Score RMSE: 0.01642800793261555
QED Score MAE: 0.013189942379447376


In [7]:
new_data= pd.DataFrame({
    "smiles": smiles,
    "QED": qed_gt, 
    "Predicted QED": pred
})
new_data.to_csv("QED_RDKit_pred.csv", index=False )

#### Experiment 5: Predict SAS score for Canonical SMILES 
- Also very high accuracy as log P predictions 

In [17]:
data= pd.read_csv("enhanced_molecules_top1000 (2).csv", usecols=[0,3])
smiles= data["smiles"]
sas_gt= data["SAS"]
pred = pred_SAS(smiles)
print("SAS Score RMSE:", RMSE(sas_gt,pred))
print("SAS Score MAE:", MAE(sas_gt,pred))

new_data= pd.DataFrame({
    "smiles": smiles,
    "SAS": sas_gt, 
    "Predicted SAS": pred
})
new_data.to_csv("SAS_RDKit_pred.csv", index=False )

SAS Score RMSE: 8.773993594040031e-16
SAS Score MAE: 5.160316618457727e-16


#### Experiment 6: QED and SAS score prediction with non-canonical SMILES 
Predict the QED scores and SAS score based on random, kekule, non-isomeric SMILES 

In [4]:
# QED Scores 
data= pd.read_csv("enhanced_molecules_top1000 (2).csv", usecols=[2,5,6,7,8])
logP_gt= data["qed"]
# all testing SMILES 
smiles_dic= {
    "ran_1": data["Random_SMILES_1"], 
    "ran_2" : data["Random_SMILES_2"], 
    "non_iso": data["NonIsomeric_SMILES"], 
    "kekule" : data["Kekule_SMILES"]
}

# running experiment to get prediction output and error 
for smiles in smiles_dic: 
    out= pred_QED(smiles_dic[smiles])
    print(f"{smiles} QED RMSE Error: {RMSE(logP_gt, out)}")
    print (f"{smiles} QED MAE Error: {MAE(logP_gt, out)}")

ran_1 QED RMSE Error: 0.016428007932615552
ran_1 QED MAE Error: 0.013189942379447374
ran_2 QED RMSE Error: 0.01642800793261555
ran_2 QED MAE Error: 0.013189942379447374
non_iso QED RMSE Error: 0.01642800793261555
non_iso QED MAE Error: 0.013189942379447376
kekule QED RMSE Error: 0.01642800793261555
kekule QED MAE Error: 0.013189942379447378


In [5]:
# SAS Scores 
data= pd.read_csv("enhanced_molecules_top1000 (2).csv", usecols=[3,5,6,7,8])
logP_gt= data["SAS"]
# all testing SMILES 
smiles_dic= {
    "ran_1": data["Random_SMILES_1"], 
    "ran_2" : data["Random_SMILES_2"], 
    "non_iso": data["NonIsomeric_SMILES"], 
    "kekule" : data["Kekule_SMILES"]
}

# running experiment to get prediction output and error 
for smiles in smiles_dic: 
    out= pred_SAS(smiles_dic[smiles])
    print(f"{smiles} SAS RMSE Error: {RMSE(logP_gt, out)}")
    print (f"{smiles} SAS MAE Error: {MAE(logP_gt, out)}")

ran_1 SAS RMSE Error: 8.773993594040031e-16
ran_1 SAS MAE Error: 5.160316618457727e-16
ran_2 SAS RMSE Error: 8.773993594040031e-16
ran_2 SAS MAE Error: 5.160316618457727e-16
non_iso SAS RMSE Error: 0.009713732467127397
non_iso SAS MAE Error: 0.00030717519177700847
kekule SAS RMSE Error: 8.773993594040031e-16
kekule SAS MAE Error: 5.160316618457727e-16


### Conclusion: 
- The error terms are repeating for corresponding experiments, this is because RDKit can parse the different SMILES into the same molecule. And the same molecule has the same error from RDKit's Descriptor Functions 